# TODO

you can't actually test this model therefore create a version with just df

In [1]:
from utils.ml import *

2023-05-19 21:15:28.629923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 21:15:28.866980: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-19 21:15:29.736751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-19 21:15:29.736933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

References:

1. https://discuss.huggingface.co/t/what-does-increasing-number-of-heads-do-in-the-multi-head-attention/1847/2

In [2]:
df_train = pd.read_excel("dataset_rnn.xlsx", sheet_name="train")
df_test = pd.read_excel("dataset_rnn.xlsx", sheet_name="test")
display(df_train)
display(df_test)

,bmg_alloy,delta_h_mix,delta_s_mix,delta_d,delta_e,actual_d_max,rnn_encoding
0,Al92Nd6Ni2,10.61,0.32,0.20,-0.17,0.00,0.00
1,Zr26Ti10Cu8Ni8Be20Y4Mg24,13.89,1.78,0.17,-0.12,5.00,5.11
2,Gd55Ni22Mn3Al20,11.82,1.09,0.26,-0.18,2.00,2.24
3,Ag53.8Mg15.4Ca30.8,10.05,0.98,0.25,-0.28,0.80,0.86
4,Ni60Nb20Zr20,19.88,0.95,0.17,-0.13,0.50,0.47
...,...,...,...,...,...,...,...
1054,Mg65Cu7.5Ni7.5Ag5Zn5Gd7.5Y2.5,9.89,1.25,0.18,-0.16,13.00,13.12
1055,Mg65Cu7.5Ni7.5Ag5Zn5Gd2.5Y7.5,9.96,1.25,0.17,-0.16,9.50,9.80
1056,Mg80Ni10Nd10,9.39,0.64,0.25,-0.09,0.60,0.28
1057,Cu45.5Mg31.8Ca22.7,10.66,1.06,0.34,-0.25,1.25,1.16


,bmg_alloy,delta_h_mix,delta_s_mix,delta_d,delta_e,actual_d_max,predicted_d_max,rnn_encoding
0,Co14Ni69P17,14.24,0.83,0.12,0.04,0.00,0.02,0.00
1,C14Fe66W20,30.81,0.87,0.14,-0.13,0.00,0.00,0.00
2,Cu15La30Mg55,8.64,0.97,0.31,-0.13,0.00,1.77,0.00
3,Al87.5Ca2.5Ni10,11.30,0.44,0.21,-0.24,0.00,-0.01,0.00
4,B36C8Co56,35.47,0.89,0.11,-0.14,0.00,-0.03,0.00
...,...,...,...,...,...,...,...,...
257,Gd55Ni25Al20,11.94,1.00,0.27,-0.19,2.00,1.36,3.91
258,Mg65Ni20La15,10.04,0.89,0.29,-0.14,0.50,0.44,2.97
259,B8Fe89Ni3,16.80,0.41,0.16,-0.02,0.00,0.02,0.00
260,La62Al14Cu18Ag6,8.44,1.05,0.36,-0.23,5.00,5.77,5.19


## 1. Test the capability of our d_max predictor

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

max_factor = max(df_train["actual_d_max"])

print(max_factor)

with open("parameters.json") as json_file:
    parameters = json.load(json_file)

30.0


In [6]:
class SequenceToSigmoid(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

def predict_d_max(alloy):
    input_size = 20
    hidden_size = 128
    learning_rate = 0.001
    num_epochs = 2000
    batch_size = 32
    num_layers = 3
    
    loaded_rnn_encoder = SequenceToSigmoid(input_size, hidden_size, num_layers)
    loaded_rnn_encoder.load_state_dict(torch.load("rnn_fully_trained.pt"))
    loaded_rnn_encoder.eval()
    
    loaded_X_transformer = pickle.load(open("catboost_X_quantiletransformer.pkl", "rb"))
    loaded_y_transformer = pickle.load(open("catboost_y_quantiletransformer.pkl", "rb"))
    loaded_catboost_predictor = pickle.load(open("catboost_model.pkl", "rb"))
    
    alloy_tensor = alloy_to_1d_tensor(alloy)
    rnn_prediction = loaded_rnn_encoder(torch.atleast_2d(alloy_tensor)).squeeze().item()
    rnn_prediction *= max_factor
    # print(rnn_prediction)
    
    h_mix, s_mix, delta_d, delta_e, max_element = prepare_params(alloy)
    # print(h_mix, s_mix, delta_d, delta_e, max_element)
    
    transformed_input = loaded_X_transformer.transform(np.array([h_mix, s_mix, delta_d, delta_e, rnn_prediction]).reshape(1, -1))
    # print(transformed_input)
    transformed_output = loaded_catboost_predictor.predict(transformed_input)
    # print(transformed_output)
    output = loaded_y_transformer.inverse_transform(transformed_output.reshape(1, -1)).item()
    return output

In [7]:
predict_d_max("Zr26Ti10Cu8Ni8Be20Y4Mg24")

5.0

## 2. Generate optimized BMGs

In [8]:
X_train = []
y_train = []

X_test = []
y_test = []

for i in range(df_train.shape[0]):
    elements, compositions = get_elements_and_compositions(df_train.loc[i, "bmg_alloy"])
    X_train.append(elements_to_1d_tensor(elements))
    y_train.append(composition_to_1d_tensor(compositions))

X_train = torch.stack(X_train)
y_train = torch.stack(y_train)

print(X_train.shape, X_train.dtype, y_train.shape, y_train.dtype)

torch.Size([1059, 10]) torch.float32 torch.Size([1059, 10]) torch.float32


In [9]:
for i in range(df_test.shape[0]):
    elements, compositions = get_elements_and_compositions(df_test.loc[i, "bmg_alloy"])
    X_test.append(elements_to_1d_tensor(elements))
    y_test.append(composition_to_1d_tensor(compositions))

X_test = torch.stack(X_test)
y_test = torch.stack(y_test)

print(X_test.shape, X_test.dtype, y_test.shape, y_test.dtype)

torch.Size([262, 10]) torch.float32 torch.Size([262, 10]) torch.float32


In [29]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()

        self.hidden_size = hidden_size

        # Encoder
        self.encoder = nn.LSTM(input_size, hidden_size)

        # Decoder
        self.decoder = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, target_seq):
        
        input_seq = input_seq.unsqueeze(1)
        target_seq = target_seq.unsqueeze(1)
        
        # Encode the input sequence
        _, hidden = self.encoder(input_seq)

        # Decode the output sequence
        output, _ = self.decoder(target_seq, hidden)
        output = self.fc(output)
        
        output = F.softmax(output, dim=2)
        output = output * (input_seq != 0).float()

        # Normalize the output sequence to sum up to 1
        output_sum = output.sum(dim=2, keepdim=True)
        output = output / output_sum
        
        return output.squeeze(1)

def custom_loss(output, target, generated_alloy_d_max, actual_alloy_d_max):
    # Calculate the loss based on the output and target
    loss = F.mse_loss(output, target)
    
    # print("loss before", loss)
    # Scale the loss based on the predicted_d_max
    
    normalized_d_max = generated_alloy_d_max / max_factor
    scaled_loss = loss / (normalized_d_max + 0.1)
    
    # return scaled_loss
    # return Variable(torch.tensor(np.power(actual_alloy_d_max - generated_alloy_d_max, 2)), requires_grad = True)
    return Variable(torch.tensor(normalized_d_max * -1), requires_grad = True)

In [30]:
input_size = alloy_max_len // 2
hidden_size = 32
output_size = alloy_max_len // 2

model = Seq2Seq(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def random_training_example(input_data, output_data):
    random_number = random.randint(0, input_data.shape[0] - 1)
    elements_tensor, compositions_tensor = input_data[random_number].view(1, -1), output_data[random_number].view(1, -1)
    
    # print(elements_tensor, compositions_tensor)
    element_input = elements_tensor[0].tolist()
    composition_output = compositions_tensor[0].tolist()
    
    elements = []
    for idx in range(len(element_input)):
        elements.append(index_to_element(element_input[idx]))
    alloy_str = ""
    
    for idx in range(min(len(composition_output), len(elements))):
        alloy_str += str(elements[idx]) + str(composition_output[idx])
    
    alloy_str = alloy_str.split("END")[0]
    # print("Actual alloy:", alloy_str)
    # print("Actual d_max:", df_train.loc[random_number, "actual_d_max"])
    
    return elements_tensor, compositions_tensor, alloy_str, df_train.loc[random_number, "actual_d_max"]

random_training_example(X_train, y_train)

(tensor([[ 5., 28., 14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 tensor([[20.0000, 77.5000,  2.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000]]),
 'B20.0Ni77.5Si2.5',
 0.0)

In [31]:
num_epochs = 3000
all_losses = []
current_loss = 0
plot_every = 100
print_every = 1000

wins_and_losses = []
better_generations = []
wins_tracker = []

for epoch in range(num_epochs):
    try:
        model.train()
        optimizer.zero_grad()
        rand_input_seq, rand_output_seq, actual_alloy, actual_alloy_d_max = random_training_example(X_train, y_train)
        output_seq = model(rand_input_seq, rand_input_seq)
        # print(np.sum(np.array(output_seq[0].tolist()) * 100))
        generated_alloy = combine_elements_and_composition_to_alloy(rand_input_seq[0].tolist(), list(np.array(output_seq[0].tolist()) * 100))
        # print("Generated Alloy", generated_alloy)
        generated_alloy_d_max = predict_d_max(generated_alloy)
        # print("Generated Alloy d_max", generated_alloy_d_max)
        # print("LOSS", loss)
        
        if generated_alloy_d_max > actual_alloy_d_max:
            wins_and_losses.append("win")
            better_generations.append(generated_alloy)
            wins_tracker.append({"actual_alloy": actual_alloy,
                                 "actual_alloy_d_max": actual_alloy_d_max,
                                 "generated_alloy": generated_alloy,
                                 "generated_alloy_d_max": generated_alloy_d_max,
                                 "gain": generated_alloy_d_max - actual_alloy_d_max
                                })
        else:
            wins_and_losses.append("loss")
        
        loss = custom_loss(output_seq, rand_output_seq, generated_alloy_d_max, actual_alloy_d_max)
        # loss = Variable(0.0, requires_grad = True) if generated_alloy_d_max > actual_d_max else Variable(1.0, requires_grad = True)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (epoch + 1) % print_every == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        if (epoch + 1) % plot_every == 0:
            all_losses.append(current_loss / plot_every)
            current_loss = 0
    except Exception as e:
        print(e)

Epoch [1000/3000], Loss: 225.0
Epoch [2000/3000], Loss: 2.25
Epoch [3000/3000], Loss: 1.0


In [32]:
from collections import Counter
Counter(wins_and_losses)

Counter({'loss': 2157, 'win': 843})

In [33]:
print(better_generations)

['Fe20.0Tb19.6B19.8Si18.9Nb21.5', 'Ca46.7Mg29.4Zn23.8', 'Au32.2Ag20.0Cu16.2Si31.5', 'Al46.8B29.3Co23.7', 'Ti23.7Zr26.0Cu23.0Ni27.1', 'Fe31.5B21.3Si21.2Zr25.7', 'Cu48.6Mg27.1Y24.1', 'Fe18.7Cr12.5Mo22.5C15.0B14.3Er16.6', 'Fe19.2Zr20.0Cu19.0Al16.6Ag24.9', 'Cu44.4Mg29.5Ca25.9', 'Ag32.4Mg18.6Ca19.6Cu29.2', 'Ag44.7Mg29.9Ca25.3', 'Ni21.0Zr11.8Ti20.4Si16.6Sn14.8Nb15.2', 'Zr44.6Ti29.8Ni25.5', 'Y34.4Fe20.8B17.9Ti26.8', 'Fe21.2Co18.2B20.0Si17.2Nb23.2', 'Pd45.2Cu30.8Si23.8', 'Fe31.0Ni23.2Y16.1B29.4', 'Ag45.1Cu30.0Fe24.7', 'Ce44.4Mg30.0Ni25.4', 'Fe17.8Mo14.7P14.5C20.0B15.9Si16.9', 'Ag45.1Cu30.0Fe24.7', 'Ca46.7Mg29.4Zn23.8', 'Al41.9Ni35.4Y22.6', 'Al46.2Cu31.5La22.1', 'Ca46.5Mg29.6Cu23.8', 'Fe20.0Tb19.6B19.8Si18.9Nb21.5', 'Ca46.7Mg29.4Zn23.8', 'Ti18.2Zr20.3Cu18.6Ni16.4Sn26.2', 'Cu26.9Ni22.0Zr18.2Ti32.7', 'Al46.2Cu31.5La22.1', 'Ag32.4Mg18.6Ca19.6Cu29.2', 'Fe56.7Zr43.2', 'Ag32.4Mg18.6Ca19.6Cu29.2', 'Fe33.7Si20.3B18.8P27.0', 'Fe19.2Co12.5Mo22.0C14.9B14.4Er16.7', 'Mg41.7Ni35.5Y22.6', 'Cu48.6Mg27.1Y24.1',

In [34]:
for i in wins_tracker:
    print(i)
    print("\n")

{'actual_alloy': 'Fe66.23999786376953Tb5.760000228881836B19.200000762939453Si4.800000190734863Nb4.0', 'actual_alloy_d_max': 3.0, 'generated_alloy': 'Fe20.0Tb19.6B19.8Si18.9Nb21.5', 'generated_alloy_d_max': 8.0, 'gain': 5.0}


{'actual_alloy': 'Ca5.0Mg85.0Zn10.0', 'actual_alloy_d_max': 0.0, 'generated_alloy': 'Ca46.7Mg29.4Zn23.8', 'generated_alloy_d_max': 2.0, 'gain': 2.0}


{'actual_alloy': 'Au46.0Ag5.0Cu29.0Si20.0', 'actual_alloy_d_max': 1.0, 'generated_alloy': 'Au32.2Ag20.0Cu16.2Si31.5', 'generated_alloy_d_max': 4.0, 'gain': 3.0}


{'actual_alloy': 'Al4.0B10.0Co86.0', 'actual_alloy_d_max': 0.0, 'generated_alloy': 'Al46.8B29.3Co23.7', 'generated_alloy_d_max': 1.0, 'gain': 1.0}


{'actual_alloy': 'Ti47.5Zr2.5Cu42.5Ni7.5', 'actual_alloy_d_max': 1.5, 'generated_alloy': 'Ti23.7Zr26.0Cu23.0Ni27.1', 'generated_alloy_d_max': 3.0, 'gain': 1.5}


{'actual_alloy': 'Fe74.25B14.850000381469727Si9.899999618530273Zr1.0', 'actual_alloy_d_max': 0.75, 'generated_alloy': 'Fe31.5B21.3Si21.2Zr25.7', 'gen